In [1]:
from pyspark.sql import SparkSession
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("CreditCard.csv")

In [4]:
data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,id
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,3039162
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,3039163
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,3039604
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,3039676
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,3039678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0,6355741
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0,6355742
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0,6355743
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0,6355744


In [5]:
data.dtypes.value_counts()

float64    30
int64       2
dtype: int64

In [6]:
features = data.drop(['Class'], axis=1)
features.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,id
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,3039162
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,3039163
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,3039604
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,3039676
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,3039678


In [7]:
target = data['Class']
target.head()

0    0
1    0
2    0
3    0
4    0
Name: Class, dtype: int64

In [67]:
type(target)

pandas.core.series.Series

In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    stratify=target, 
                                                    test_size=0.3)

In [9]:
X_test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,id
13746,24380.0,1.113860,-0.562985,1.039270,1.000011,-1.227966,-0.132527,-0.798634,0.037943,0.862177,...,-0.459053,-0.634747,0.066856,0.262326,0.236697,-0.366628,0.040353,0.035697,70.30,2804161
181019,124789.0,-1.257555,1.467700,-0.121133,-1.120809,1.067942,-0.490209,1.858603,-0.831433,1.320759,...,-0.568169,-0.238972,-0.097693,0.604524,-0.605337,0.010413,-0.006149,-0.540230,42.12,2992757
6471,7739.0,-2.608460,-0.980463,2.577244,-0.253370,2.858780,-0.949880,1.194871,-2.043584,2.885661,...,-0.838785,0.084507,-1.529359,0.308810,0.113584,0.164855,-1.808663,-1.447678,29.99,728675
127462,78339.0,-0.926879,-0.055293,-0.058322,-1.556800,1.207941,4.004034,-0.418578,1.417161,-1.328016,...,-0.074280,-0.144958,0.121874,0.954398,0.167027,-0.402425,0.007909,0.082832,135.00,7286401
119877,75618.0,-1.749060,1.542895,0.004060,-0.156701,0.049081,-0.912730,0.670277,0.101715,-0.114614,...,-0.021562,0.049591,-0.259285,-0.065833,-0.119085,0.273660,0.062845,0.285950,40.25,7873091


In [23]:
X_test.shape

(85443, 31)

In [10]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [16]:
import onnxruntime as rt
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn
import numpy

In [35]:
initial_type = [("float_input", FloatTensorType([None, 31]))]
onx = convert_sklearn(rf, initial_types=initial_type, target_opset=21)

with open("credit_card_model.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [30]:
X_test_np = numpy.array(X_test)

In [37]:
sess = rt.InferenceSession("credit_card_model.onnx", providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test_np.astype(numpy.float32)})[0]
print(pred_onx)

[0 0 0 ... 0 0 0]


In [40]:
# Load the ONNX model file in binary mode
with open("credit_card_model.onnx", "rb") as f:
    onnx_binary = f.read()

# Convert the binary data to hexadecimal
onnx_hex = onnx_binary.hex()

# Save the hexadecimal representation to a .hex file
with open("credit_card_model.onnx.hex", "w") as hex_file:
    hex_file.write(onnx_hex)

print("ONNX model has been successfully converted to a .hex file.")



ONNX model has been successfully converted to a .hex file.
